In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

In [96]:
steam_description_df = pd.read_csv("steam_description_data.csv")
steam_media_df = pd.read_csv("steam_media_data.csv")
steam_requirements_df = pd.read_csv("steam_requirements_data.csv")
steam_support_df = pd.read_csv("steam_support_info.csv")
steam_user = pd.read_csv("steam_user_behavior.csv")
steam_df = pd.read_csv("steam.csv")
steamspy_tag_df = pd.read_csv("steamspy_tag_data.csv")

In [97]:
# Crear las columnas "purchase" y "play"
purchase_data = steam_user[steam_user['Action'] == 'purchase'].copy()
play_data = steam_user[steam_user['Action'] == 'play'].copy()

# Agregar la columna "_purchase" y "_play"
purchase_data['Value_purchase'] = 1
play_data['Value_play'] = play_data['Value']

# Fusionar los datos
merged_data = pd.merge(purchase_data, play_data, on=['UserID', 'Name'], how='left')
merged_data['Value_play'].fillna(0, inplace=True)

# Eliminar columnas innecesarias
merged_data.drop(['Action_x', 'Action_y', 'Value_x', 'Value_y'], axis=1, inplace=True)
steam_user_clean = merged_data.drop('Value_purchase', axis=1)


# Ver el resultado
print(steam_user_clean.head())

      UserID                        Name  Value_play
0  151603712  The Elder Scrolls V Skyrim       273.0
1  151603712                   Fallout 4        87.0
2  151603712                       Spore        14.9
3  151603712           Fallout New Vegas        12.1
4  151603712               Left 4 Dead 2         8.9


from sklearn.preprocessing import LabelEncoder

#Crear una instancia del LabelEncoder
encoder = LabelEncoder()

#Codificar la columna "Name"
steam_user_clean['Name_encoded'] = encoder.fit_transform(steam_user_clean['Name'])

#Eliminar la columna "Name"
steam_user_clean.drop('Name', axis=1, inplace=True)

# Reorganizar las columnas
steam_user_clean = steam_user_clean[['UserID', 'Name_encoded', 'Value_play']]

print(steam_user_clean.head())

In [98]:
from scipy.stats import percentileofscore

# Calcular el percentil de las horas jugadas
percentiles = steam_user_clean['Value_play'].apply(lambda x: percentileofscore(steam_user_clean['Value_play'], x))

# Mapear los percentiles a un rango de 0 a 100
rating = percentiles.apply(lambda x: int(x * 100 / 100))

# Agregar la columna "Rating" al DataFrame
steam_user_clean['Rating'] = rating

In [99]:
steam_user_clean.head(100)
steam_user_clean.to_csv('steam_user_clean.csv', index=False)

In [100]:
# Cambiar el nombre de la columna "appid" a "steam_appid"
steam_df.rename(columns={"appid": "steam_appid"}, inplace=True)
steamspy_tag_df.rename(columns={"appid": "steam_appid"}, inplace=True)

In [101]:
#Combinamos los datasets de la info de los videojuegos
games_df = steam_description_df.merge(steam_media_df, on="steam_appid").merge(steam_requirements_df, on="steam_appid").merge(steam_support_df, on="steam_appid").merge(steam_df, on="steam_appid").merge(steamspy_tag_df, on="steam_appid")

In [102]:
# Crear un nuevo DataFrame con columnas seleccionadas
games_df_clean = games_df[['steam_appid', 'short_description', 'name','release_date', 'developer', 'platforms','categories', 'genres', 'steamspy_tags','price']]

In [103]:
print(steam_user_clean)

           UserID                        Name  Value_play  Rating
0       151603712  The Elder Scrolls V Skyrim       273.0      98
1       151603712                   Fallout 4        87.0      95
2       151603712                       Spore        14.9      84
3       151603712           Fallout New Vegas        12.1      82
4       151603712               Left 4 Dead 2         8.9      79
...           ...                         ...         ...     ...
129530  128470551                Fallen Earth         2.4      66
129531  128470551                 Magic Duels         2.2      65
129532  128470551                 Titan Souls         1.5      62
129533  128470551  Grand Theft Auto Vice City         1.5      62
129534  128470551                        RUSH         1.4      61

[129535 rows x 4 columns]


In [104]:
games_df_clean.rename(columns={"name": "Name"}, inplace=True)
print(games_df_clean)

       steam_appid                                  short_description   
0               10  Play the world's number 1 online action game. ...  \
1               30  Enlist in an intense brand of Axis vs. Allied ...   
2               50  Return to the Black Mesa Research Facility as ...   
3               70  Named Game of the Year by over 50 publications...   
4               80  With its extensive Tour of Duty campaign, a ne...   
...            ...                                                ...   
26863      1065230  The Room of Pandora is a third-person interact...   
26864      1065570  Cyber Gun is a hardcore first-person shooter w...   
26865      1065650  Super Star Blast is a space based game with ch...   
26866      1066700  Pursue a snow-white deer through an enchanted ...   
26867      1069460  A portal has opened and dark magic is pouring ...   

                                 Name release_date           developer   
0                      Counter-Strike   2000-11-0

C:\Users\emili\AppData\Local\Temp\ipykernel_17640\3477535913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df_clean.rename(columns={"name": "Name"}, inplace=True)


In [105]:
import re

# Eliminar caracteres no alfabéticos de los nombres de los videojuegos
steam_user_clean['Name'] = steam_user_clean['Name'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
games_df_clean['Name'] = games_df_clean['Name'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

# Eliminar dobles espacios
steam_user_clean['Name'] = steam_user_clean['Name'].apply(lambda x: ' '.join(x.split()))
games_df_clean['Name'] = games_df_clean['Name'].apply(lambda x: ' '.join(x.split()))

# Capitalizar la primera letra de cada palabra
steam_user_clean['Name'] = steam_user_clean['Name'].apply(lambda x: x.title())
games_df_clean['Name'] = games_df_clean['Name'].apply(lambda x: x.title())


steam_user_clean.to_csv('steam_user_clean.csv', index=False)
games_df_clean.to_csv('games_df_clean.csv', index=False)

C:\Users\emili\AppData\Local\Temp\ipykernel_17640\4155422256.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df_clean['Name'] = games_df_clean['Name'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
C:\Users\emili\AppData\Local\Temp\ipykernel_17640\4155422256.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df_clean['Name'] = games_df_clean['Name'].apply(lambda x: ' '.join(x.split()))
C:\Users\emili\AppData\Local\Temp\ipykernel_17640\4155422256.py:13: SettingWithCopyWarning: 
A value is

In [106]:
#games_df_clean.to_csv('games_df_clean.csv', index=False)
games_df_clean.head()

,steam_appid,short_description,Name,release_date,developer,platforms,categories,genres,steamspy_tags,price
0,10,Play the world's number 1 online action game. ...,Counterstrike,2000-11-01,Valve,windows;mac;linux,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,7.19
1,30,Enlist in an intense brand of Axis vs. Allied ...,Day Of Defeat,2003-05-01,Valve,windows;mac;linux,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,3.99
2,50,Return to the Black Mesa Research Facility as ...,Halflife Opposing Force,1999-11-01,Gearbox Software,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,3.99
3,70,Named Game of the Year by over 50 publications...,Halflife,1998-11-08,Valve,windows;mac;linux,Single-player;Multi-player;Online Multi-Player...,Action,FPS;Classic;Action,7.19
4,80,"With its extensive Tour of Duty campaign, a ne...",Counterstrike Condition Zero,2004-03-01,Valve,windows;mac;linux,Single-player;Multi-player;Valve Anti-Cheat en...,Action,Action;FPS;Multiplayer,7.19


In [114]:
correlation_matrix_users = steam_user_clean.corr()

ValueError: could not convert string to float: 'The Elder Scrolls V Skyrim'

In [112]:
# Combinar los DataFrames basándose en la columna "Name"
df = 222steam_user_clean.merge(games_df_clean, on='Name', how='left')

In [113]:
df.dropna(inplace=True)
#Reorganizar columnas
df = df[[col for col in df.columns if col != 'Rating'] + ['Rating']]
df.head()

,UserID,Name,Value_play,steam_appid,short_description,release_date,developer,platforms,categories,genres,steamspy_tags,price,Rating
0,151603712,The Elder Scrolls V Skyrim,273.0,72850.0,EPIC FANTASY REBORN The next chapter in the hi...,2011-11-10,Bethesda Game Studios,windows,Single-player;Steam Achievements;Steam Trading...,RPG,Open World;RPG;Fantasy,9.99,98
1,151603712,Fallout 4,87.0,377160.0,"Bethesda Game Studios, the award-winning creat...",2015-11-09,Bethesda Game Studios,windows,Single-player;Steam Achievements;Full controll...,RPG,Open World;Post-apocalyptic;Exploration,19.99,95
2,151603712,Spore,14.9,17390.0,"From Single Cell to Galactic God, evolve your ...",2008-12-19,Maxis™,windows,Single-player,Simulation,Simulation;Sandbox;Open World,14.99,84
3,151603712,Fallout New Vegas,12.1,22380.0,Welcome to Vegas. New Vegas. Enjoy your stay!,2010-10-21,Obsidian Entertainment,windows,Single-player;Steam Achievements;Partial Contr...,Action;RPG,Open World;RPG;Post-apocalyptic,7.99,82
4,151603712,Left 4 Dead 2,8.9,550.0,"Set in the zombie apocalypse, Left 4 Dead 2 (L...",2009-11-19,Valve,windows;mac;linux,Single-player;Multi-player;Co-op;Steam Achieve...,Action,Zombies;Co-op;FPS,7.19,79


In [ ]:
columns_to_clean = ["short_description","release_date","developer","platforms","categories","genres","steamspy_tags"]